In [59]:
import cv2
import pandas as pd
import numpy as np

In [60]:
df_1 = pd.read_csv("Slideseq_1_glomeruli_position.txt", sep="\t", header=0, index_col=0)
df_2 = pd.read_csv("Slideseq_2_glomeruli_position.txt", sep="\t", header=0, index_col=0)
df_1["AP"] = df_1["AP"] / 4000
df_2["AP"] = df_2["AP"] / 4000
df_1.sort_values("OR", inplace=True)
df_2.sort_values("OR", inplace=True)
print(df_1.head())
print(df_2.head())

          OR  Slide  Average           ML           DV    AP
31  Olfr1000      7     4.00  -771.154506 -1973.983747   7.0
56  Olfr1018     10     4.75  -718.368031 -2787.781019  10.0
46  Olfr1030      9     4.75  -653.009496 -1383.292778   9.0
47  Olfr1031      9     4.50  -710.608633 -1251.281594   9.0
57  Olfr1045     10     5.00 -1295.912350 -1593.913807  10.0
          OR  Slide  Average           ML           DV    AP
32  Olfr1000     11     5.00 -1674.094228 -1967.999543  11.0
73  Olfr1013     17     4.75 -1149.473539 -1449.529952  17.0
14  Olfr1030      9     4.50 -1015.996164 -1303.241413   9.0
33  Olfr1048     11     4.50  -807.635396 -1108.751969  11.0
40  Olfr1086     12     4.25 -1155.866224  -311.059204  12.0


In [61]:
ORs_1 = df_1.OR.unique()
ORs_2 = df_2.OR.unique()
shared_ORs = [x for x in ORs_1 if x in ORs_2]
df_1 = df_1[df_1["OR"].isin(shared_ORs)].reset_index(drop=True)
df_2 = df_2[df_2["OR"].isin(shared_ORs)].reset_index(drop=True)
#coords_1['rep']=1
#coords_2['rep']=2
#Frame = coords_1.append(pd.DataFrame(data = coords_2), ignore_index=True)

In [62]:
n = len(shared_ORs)
for z in range(n):
    OR_hold = df_1["OR"][z]
    tmpdf1 = df_1.drop(z)
    tmpdf2 = df_2.drop(z)
    coords_1 = tmpdf1[["AP", "DV"]].values
    coords_2 = tmpdf2[["AP", "DV"]].values
    H, _ = cv2.findHomography(coords_1, coords_2)
    n = coords_2.shape[0]
    new_coords_1 = np.zeros((n, 3))
    for i in range(n):
        pt1 = np.array([coords_1[i,0], coords_1[i,1], 1])
        pt1 = pt1.reshape(3, 1)
        pt2 = np.dot(H, pt1)
        pt2 = pt2/pt2[2]
        new_coords_1[i,0] = pt2[0]
        new_coords_1[i,1] = pt2[1]
    with open("transformed_coords_1to2_"+OR_hold+".txt", "w") as f:
        for index, row in df_2.iterrows():
            print(row["OR"], row["AP"], row["DV"], 2, file=f, sep="\t")
        for index, row in df_1.iterrows():
            pt1 = np.array([row["AP"], row["DV"], 1])
            pt1 = pt1.reshape(3, 1)
            pt2 = np.dot(H, pt1)
            pt2 = pt2/pt2[2]
            print(row["OR"], pt2[0,0], pt2[1,0], 1, file=f, sep="\t")

In [55]:
df_2

,OR,Slide,Average,ML,DV,AP
32,Olfr1000,11,5.00,-1674.094228,-1967.999543,11.0
73,Olfr1013,17,4.75,-1149.473539,-1449.529952,17.0
14,Olfr1030,9,4.50,-1015.996164,-1303.241413,9.0
33,Olfr1048,11,4.50,-807.635396,-1108.751969,11.0
40,Olfr1086,12,4.25,-1155.866224,-311.059204,12.0
49,Olfr1123,13,4.75,-1176.187204,-3356.905358,13.0
61,Olfr1189,15,4.25,-1739.883966,-2444.896994,15.0
62,Olfr1217,15,5.00,-1199.584554,-2417.357228,15.0
67,Olfr122,16,4.00,-190.969569,-3542.786412,16.0
74,Olfr1229,17,4.00,-541.492221,-4222.876533,17.0


In [38]:
x

0    Olfr1000
Name: OR, dtype: object